Pobierz dane dotyczące scoringu kredytowego

*Pracujesz jako analityk danych w globalnej firmie finansowej. Przez lata firma gromadziła podstawowe dane bankowe i wiele informacji związanych z kredytami. Kierownictwo chce zbudować inteligentny system do segregowania ludzi w przedziałach oceny kredytowej, aby zmniejszyć wysiłek ręczny.*

**Zadanie na dziś: przygotować dane do modelu**

In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

import danych

In [2]:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "train.csv"

# Load the latest version
df = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS, "parisrohan/credit-score-classification", file_path,
)

print("First 5 records:", df.head())

<ipython-input-2-db52f5663265>:9: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS, "parisrohan/credit-score-classification", file_path,


100%|██████████| 6.09M/6.09M [00:00<00:00, 15.6MB/s]

Extracting zip of train.csv...



/usr/local/lib/python3.11/dist-packages/kagglehub/pandas_datasets.py:91: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(


First 5 records:        ID Customer_ID     Month           Name   Age          SSN Occupation  \
0  0x1602   CUS_0xd40   January  Aaron Maashoh    23  821-00-0265  Scientist   
1  0x1603   CUS_0xd40  February  Aaron Maashoh    23  821-00-0265  Scientist   
2  0x1604   CUS_0xd40     March  Aaron Maashoh  -500  821-00-0265  Scientist   
3  0x1605   CUS_0xd40     April  Aaron Maashoh    23  821-00-0265  Scientist   
4  0x1606   CUS_0xd40       May  Aaron Maashoh    23  821-00-0265  Scientist   

  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  ...  Credit_Mix  \
0      19114.12            1824.843333                  3  ...           _   
1      19114.12                    NaN                  3  ...        Good   
2      19114.12                    NaN                  3  ...        Good   
3      19114.12                    NaN                  3  ...        Good   
4      19114.12            1824.843333                  3  ...        Good   

   Outstanding_Debt Credit_Utiliz

**Poznaj dane:**

1.   sprawdź wymiary i typy kolumn
2.   usuń duplikaty
3.   usuń zdbędne kolumny
4.   sprawdz liczbę braków danych w poszczególnych kolumnach






In [3]:
# Sprawdzenie wymiarów danych
print("Wymiary zbioru danych:", df.shape)

# Sprawdzenie typów danych
print("\nTypy kolumn:\n", df.dtypes)

# Usunięcie duplikatów
df = df.drop_duplicates()
print("\nLiczba rekordów po usunięciu duplikatów:", df.shape[0])

# Usunięcie zbędnych kolumn (możesz dodać konkretne kolumny do usunięcia)
zbędne_kolumny = []  # np. ["ID", "Timestamp"]
df = df.drop(columns=zbędne_kolumny, errors='ignore')
print("\nKolumny po usunięciu zbędnych:", df.columns)

# Sprawdzenie braków danych
print("\nLiczba braków danych w kolumnach:\n", df.isnull().sum())


Wymiary zbioru danych: (100000, 28)

Typy kolumn:
 ID                           object
Customer_ID                  object
Month                        object
Name                         object
Age                          object
SSN                          object
Occupation                   object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Payment_Behaviour            object
Monthly_Balan

Usuń kolumny, które nie będą wykorzystywane w analizie ['ID','Customer_ID','SSN','Name']

In [4]:
# Lista kolumn do usunięcia
kolumny_do_usuniecia = ['ID', 'Customer_ID', 'SSN', 'Name']

# Usunięcie kolumn
df = df.drop(columns=kolumny_do_usuniecia, errors='ignore')

# Sprawdzenie, czy kolumny zostały usunięte
print("\nKolumny po usunięciu zbędnych:", df.columns)


Kolumny po usunięciu zbędnych: Index(['Month', 'Age', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary',
       'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan',
       'Type_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment',
       'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Credit_Mix',
       'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')


usuń niektore z zbędnych znakow ('_','-'), zmień puste komorki na nan

In [5]:
import numpy as np

# Usunięcie znaków '_' i '-' z nazw kolumn
df.columns = df.columns.str.replace(r'[_\-]', '', regex=True)

# Zamiana pustych wartości ("", " ") na NaN
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

# Sprawdzenie wyników
print("\nNowe nazwy kolumn:", df.columns)
print("\nLiczba braków danych po zamianie pustych komórek na NaN:\n", df.isnull().sum())


Nowe nazwy kolumn: Index(['Month', 'Age', 'Occupation', 'AnnualIncome', 'MonthlyInhandSalary',
       'NumBankAccounts', 'NumCreditCard', 'InterestRate', 'NumofLoan',
       'TypeofLoan', 'Delayfromduedate', 'NumofDelayedPayment',
       'ChangedCreditLimit', 'NumCreditInquiries', 'CreditMix',
       'OutstandingDebt', 'CreditUtilizationRatio', 'CreditHistoryAge',
       'PaymentofMinAmount', 'TotalEMIpermonth', 'Amountinvestedmonthly',
       'PaymentBehaviour', 'MonthlyBalance', 'CreditScore'],
      dtype='object')

Liczba braków danych po zamianie pustych komórek na NaN:
 Month                         0
Age                           0
Occupation                    0
AnnualIncome                  0
MonthlyInhandSalary       15002
NumBankAccounts               0
NumCreditCard                 0
InterestRate                  0
NumofLoan                     0
TypeofLoan                11408
Delayfromduedate              0
NumofDelayedPayment        7002
ChangedCreditLimit            0


Zmień nazwy miesiecy na zmienne liczbowe zgodnie z kolejnością miesięcy

In [6]:
month_mapping = {
    "January": 1, "February": 2, "March": 3, "April": 4, "May": 5, "June": 6,
    "July": 7, "August": 8, "September": 9, "October": 10, "November": 11, "December": 12
}

df['Month'] = df['Month'].map(month_mapping)

# Sprawdzenie wyników
print(df[['Month']].head())



   Month
0      1
1      2
2      3
3      4
4      5


Zakoduj zmienne tekstowe: 'Occupation', 'Type_of_Loan', 'Payment_Behaviour', 'Credit_Mix', użyj LabelEncodera

In [8]:
from sklearn.preprocessing import LabelEncoder

# Lista kolumn do zakodowania
kolumny_do_kodowania = ['Occupation', 'TypeofLoan', 'PaymentBehaviour', 'CreditMix']

# Inicjalizacja LabelEncodera
le = LabelEncoder()

# Kodowanie każdej kolumny z listy
for col in kolumny_do_kodowania:
    df[col] = le.fit_transform(df[col])

# Sprawdzenie wyników
print(df[kolumny_do_kodowania].head())


   Occupation  TypeofLoan  PaymentBehaviour  CreditMix
0          12         128                 3          3
1          12         128                 4          1
2          12         128                 5          1
3          12         128                 6          1
4          12         128                 2          1


Przelicz kolumne Credit_History_Age na liczbę mięsiecy


In [9]:
import re

# Funkcja konwertująca wiek historii kredytowej na miesiące
def convert_to_months(value):
    if pd.isna(value):  # Obsługa wartości NaN
        return np.nan
    match = re.match(r"(\d+)\s*Years?\s*(\d*)\s*Months?", str(value))
    if match:
        years = int(match.group(1))
        months = int(match.group(2)) if match.group(2) else 0
        return years * 12 + months
    return np.nan  # Jeśli format jest niepoprawny, zwróć NaN

# Przekształcenie kolumny
df['CreditHistoryAge'] = df['CreditHistoryAge'].apply(convert_to_months)

# Sprawdzenie wyników
print(df[['CreditHistoryAge']].head())


   CreditHistoryAge
0               NaN
1               NaN
2               NaN
3               NaN
4               NaN


In [10]:
# Rozdzielanie wartości po przecinku i usuwanie zbędnych spacji
df['TypeofLoan'] = df['TypeofLoan'].str.split(', ')

# Użycie MultiLabelBinarizer do One-Hot Encoding
mlb = MultiLabelBinarizer()
encoded_df = pd.DataFrame(mlb.fit_transform(df['TypeofLoan']), columns=mlb.classes_)

# Łączenie z oryginalnym DataFrame
df = df.join(encoded_df).drop(columns=['TypeofLoan'])

print(df.head())

AttributeError: Can only use .str accessor with string values!

Zmien inne zmienne na numeryczne

Uzupełnij braki w kolumnie Monthly_Inhand_Salary,  Annual_Income, .. średnią

Narysuj rozkłady zmiennych 'Annual_Income', 'Monthly_Balance'

usuń wartości odstające (w przypadku wieku wszystkie ujemne i te powyzej 100 lat)

Zwizualizuj korelacje między zmiennymi